In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import spearmanr as scor
from scipy.stats import mannwhitneyu as mwu

Importing the Niche Net matrix

In [3]:
nn_model=pd.read_csv('../data/ligand_target_matrix.csv', sep=',', header=0, index_col=0)

In [4]:
nn_model.head()

,CXCL1,CXCL2,CXCL3,CXCL5,PPBP,CXCL6,CXCL8,CXCL9,CXCL10,CXCL11,...,PCDHGB1,CDH6,CDH9,CDH12,CDH18,PCDHB16,PCDHA7,PCDHA6,PCDHA4,PCDHA12
A1BG,0.000353,0.000404,0.000373,0.000308,0.000263,0.000273,0.000414,0.000298,0.000416,0.000308,...,0.000303,0.000319,0.000323,0.000348,0.000393,0.000334,0.000335,0.000288,0.000366,0.000323
A1BG-AS1,0.000165,0.000151,0.000158,0.000132,0.000123,0.000118,0.000217,0.000141,0.000170,0.000137,...,0.000160,0.000164,0.000164,0.000178,0.000211,0.000145,0.000166,0.000140,0.000184,0.000175
A1CF,0.000579,0.000460,0.000390,0.000329,0.000321,0.000309,0.000483,0.000370,0.000393,0.000358,...,0.000425,0.001664,0.001589,0.000940,0.001422,0.000432,0.000575,0.000536,0.000517,0.000403
A2M,0.000603,0.000600,0.000516,0.000452,0.000459,0.000416,0.000703,0.000541,0.000616,0.000497,...,0.000473,0.000508,0.000497,0.000587,0.000570,0.000514,0.000536,0.000497,0.000572,0.000554
A2M-AS1,0.000089,0.000082,0.000075,0.000049,0.000051,0.000048,0.000138,0.000055,0.000061,0.000080,...,0.000032,0.000078,0.000077,0.000079,0.000080,0.000033,0.000081,0.000005,0.000050,0.000080


Getting the single drugs used in CREEDs 

In [23]:
drug_perturb=pd.read_csv('../data/single_drug_perturbations-v1.0.csv', sep=',', header=0, index_col=0)

In [33]:
drugs=list(drug_perturb['drug_name'].unique())

In [34]:
fout=open ('../data/drugs_perturb.txt', 'w') 
for line in drugs:
    fout.write(line + '\n')
fout.close()

Importing single perturbed genes from CREEDs

In [19]:
gene_perturbed=pd.read_csv('../data/single_gene_perturbations-v1.0.csv', sep=',', header=0, index_col=0)

In [21]:
gene_perturbed.head()

,cell_type,chdir_norm,ctrl_ids,curator,geo_id,hs_gene_symbol,mm_gene_symbol,organism,pert_ids,pert_type,platform,version
id,,,,,,,,,,,,
gene:508,T reg,NaN,GSM998922|GSM998923,dsurujon,GSE40655,FOXO1,Foxo1,mouse,GSM998920|GSM998921,KO,GPL1261,1.0
gene:509,heart,NaN,GSM1080530|GSM1080531|GSM1080532|GSM1080533,dsurujon,GSE44192,PLIN5,Plin5,mouse,GSM1080534|GSM1080535|GSM1080536|GSM1080537,OE,GPL6246,1.0
gene:2349,Inducible stable HEK 293 Flp-In T-REx cells ov...,NaN,GSM997725|GSM997726|GSM997727,gszeto,GSE40601,ERO1L,Ero1l,human,GSM997728|GSM997729|GSM997730,overexpressing a hyperactive mutant,GPL6244,1.0
gene:2348,Inducible stable HEK 293 Flp-In T-REx Ero1alph...,NaN,GSM997719|GSM997720|GSM997721,gszeto,GSE40601,ERO1L,Ero1l,human,GSM997722|GSM997723|GSM997724,OE,GPL6244,1.0
gene:2343,OT-I T cells from spleen,NaN,GSM1290674|GSM1290675|GSM1290676,AliDot,GSE53388,PPP2R2D,Ppp2r2d,mouse,GSM1290677|GSM1290678|GSM1290679,KD,GPL1261,1.0


In [22]:
gene_perturbed.shape

(2176, 12)